In [ ]:
! pip install osmnx

In [13]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import contextily as ctx
from shapely.geometry import Point, LineString
import osmnx as ox

In [ ]:
###USING DATASET CONTAINING POLYGONS FOR 15 BELGIAN CITIES

cities = gpd.read_file('Belgium Urban Polygons/BE_STATBEL_SH_SU_UA_CITY/BE_STATBEL_SH_SU_UA_CITY_2019_v60.gpkg')
cities.head(15)

In [78]:
# Only keeping some of the cities
objectid_list = [1, 2, 3, 4, 5, 6, 8, 11]  
cities = cities[cities['OBJECTID'].isin(objectid_list)]

city_name_mapping = {
    'Bruxelles / Brussel (greater city)': 'Brussels',
    'Charleroi (greater city)': 'Charleroi',
    'Liège (greater city)': 'Liege'
}

cities['CityName'] = cities['CityName'].replace(city_name_mapping)
cities.head(10)

,OBJECTID,CityCod,CityName,NameLg,CntryCod,hm²_3812,hm²_3035,ValidFrom,ValidTo,geometry
0,1,BE001K1,Brussels,French / Dutch,BE,16242.379883,16244.017619,2019-01-01,None,"POLYGON ((4.40633 50.91309, 4.40722 50.91295, ..."
1,2,BE002C1,Antwerpen,Dutch,BE,20442.529297,20441.663268,2019-01-01,None,"POLYGON ((4.39973 51.15106, 4.39896 51.15086, ..."
2,3,BE003C1,Gent,Dutch,BE,15774.230469,15774.773400,2019-01-01,None,"POLYGON ((3.71110 50.97954, 3.71064 50.97991, ..."
3,4,BE004K1,Charleroi,French,BE,13030.849609,13032.581282,2019-01-01,None,"POLYGON ((4.46036 50.49013, 4.45962 50.48971, ..."
4,5,BE005K1,Liege,French,BE,21262.189453,21264.940140,2019-01-01,None,"POLYGON ((5.49902 50.72076, 5.49926 50.72072, ..."
5,6,BE006C1,Brugge,Dutch,BE,14090.059570,14089.551774,2019-01-01,None,"POLYGON ((3.21374 51.16511, 3.21330 51.16501, ..."
7,8,BE008C1,Leuven,Dutch,BE,5750.669922,5751.191709,2019-01-01,None,"POLYGON ((4.74200 50.84267, 4.74186 50.84270, ..."
10,11,BE011C1,Oostende,Dutch,BE,4095.780029,4095.708565,2019-01-01,None,"POLYGON ((2.83850 51.20215, 2.84002 51.20279, ..."


In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
cities.plot(ax=ax, color='blue', edgecolor='black')

# Add basemap
ctx.add_basemap(ax, crs=cities.crs.to_string())

ax.set_title('Belgian Urban Areas')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

plt.show()


In [80]:
##SETTING THE CRS CORRECTLY

if cities.crs is None or cities.crs.to_string() != 'EPSG:4326':
    cities = cities.to_crs(epsg=4326)

# Verify the CRS
print(cities.crs)

EPSG:4326


In [81]:
# Function to obtain all streets in urban polygons
def get_streets_within_polygon(polygon):
    bbox = polygon.bounds
    # Fetch streets within the bounding box
    streets = ox.graph_from_bbox(north=bbox[3], south=bbox[1], east=bbox[2], west=bbox[0], network_type='all')
    # Convert the graph to a GeoDataFrame
    streets_gdf = ox.graph_to_gdfs(streets, nodes=False, edges=True)
    # Filter streets to those lie within the polygon
    streets_within_polygon = streets_gdf[streets_gdf.intersects(polygon)]
    return streets_within_polygon

# Function to sample points from streets
def sample_points_on_streets(edges, num_points=4):
    sampled_points = []
    for _, row in edges.iterrows():
        line = row.geometry
        if isinstance(line, LineString):
            length = line.length
            distances = np.linspace(0, length, num_points)
            points = [line.interpolate(distance) for distance in distances]
            sampled_points.extend(points)
    return sampled_points



In [55]:
##DEFINING POLYGON, STREETS, SAMPLE POINTS
leuven_polygon = cities[cities['CityName'] == 'Leuven'].geometry.iloc[0]
leuven_streets = get_streets_within_polygon(leuven_polygon)
leuven_points = sample_points_on_streets(leuven_streets, num_points=3)
leuven_points_gdf = gpd.GeoDataFrame(geometry=leuven_points, crs=leuven_streets.crs)

##PLOTTING RESULTS
fig, ax = plt.subplots(figsize=(10, 10))
leuven_streets.plot(ax=ax, color='gray', zorder=1)
leuven_points_gdf.plot(ax=ax, color='red', markersize=.05, zorder=2, alpha=.3)  # Adjust zorder and markersize here

ax.set_title('Sampled Points on Streets')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.show()

/var/folders/_7/5067xfmn3k19lcbv0zbwb5xm0000gn/T/ipykernel_50764/2105805394.py:8: FutureWarning: The `north`, `south`, `east`, and `west` parameters are deprecated and will be removed in the v2.0.0 release. Use the `bbox` parameter instead. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  streets = ox.graph_from_bbox(north=bbox[3], south=bbox[1], east=bbox[2], west=bbox[0], network_type='all')


In [ ]:
# Iterate over each city
for city_name, city_polygon in cities[['CityName', 'geometry']].values:
    # Name the polygon object according to the city
    exec(f"{city_name.lower()}_polygon = city_polygon")
    
    # Fetch streets within the city polygon and name the object accordingly
    streets = get_streets_within_polygon(city_polygon)
    exec(f"{city_name.lower()}_streets = streets")
    
    # Sample points on the streets and name the object accordingly
    points = sample_points_on_streets(streets, num_points=3)
    exec(f"{city_name.lower()}_points = points")
    
    # Convert points to a GeoDataFrame and name the object accordingly
    points_gdf = gpd.GeoDataFrame(geometry=points, crs=streets.crs)
    exec(f"{city_name.lower()}_points_gdf = points_gdf")
    
    # Plot the results
    fig, ax = plt.subplots(figsize=(10, 10))
    streets.plot(ax=ax, color='gray', zorder=1)
    points_gdf.plot(ax=ax, color='red', markersize=.05, zorder=2, alpha=.3)
    
    ax.set_title(f'Sampled Points on Streets - {city_name}')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.show()


In [86]:
##Reading in the clean data
aeds = gpd.read_file('Data/aeds.csv')
cards = gpd.read_file('Data/cards.csv')
vehicles = gpd.read_file('Data/vehicles.csv')

In [89]:
aeds['latitude'] = pd.to_numeric(aeds['latitude'], errors='coerce')
aeds['longitude'] = pd.to_numeric(aeds['longitude'], errors='coerce')

cards['latitude'] = pd.to_numeric(cards['latitude'], errors='coerce')
cards['longitude'] = pd.to_numeric(cards['longitude'], errors='coerce')

vehicles['latitude'] = pd.to_numeric(vehicles['latitude'], errors='coerce')
vehicles['longitude'] = pd.to_numeric(vehicles['longitude'], errors='coerce')

In [ ]:
def filter_points_within_polygon(points, polygon, lat_column='latitude', lon_column='longitude'):
    # Create a Shapely Point geometry column from latitude and longitude
    points['geometry'] = gpd.points_from_xy(points[lon_column], points[lat_column])
    
    # Convert the points DataFrame to a GeoDataFrame
    gdf_points = gpd.GeoDataFrame(points, geometry='geometry')
    
    # Filter points within the polygon
    filtered_points = gdf_points[gdf_points.geometry.within(polygon)]
    
    return filtered_points

##Creating aed, card and vehicle datasets for each city
for city_name, city_polygon in cities[['CityName', 'geometry']].values:
    #aeds
    city_aeds = filter_points_within_polygon(aeds, city_polygon)
    print(f"Number of AEDs in {city_name}: {len(city_aeds)}")
    city_aeds.to_csv(f'Data/{city_name}_aeds.csv', index=False)
    
    #cards
    city_cards = filter_points_within_polygon(cards, city_polygon)
    print(f"Number of cards in {city_name}: {len(city_cards)}")
    city_cards.to_csv(f'Data/{city_name}_cards.csv', index=False)
    
    #vehicles
    city_vehicles = filter_points_within_polygon(vehicles, city_polygon)
    print(f"Number of vehicles in {city_name}: {len(city_vehicles)}")
    city_vehicles.to_csv(f'Data/{city_name}_vehicles.csv', index=False)


In [ ]:
for city_name, city_polygon in cities[['CityName', 'geometry']].values:
    # Filter points within the city polygon
    city_aeds = filter_points_within_polygon(aeds, city_polygon)
    city_cards = filter_points_within_polygon(cards, city_polygon)
    city_vehicles = filter_points_within_polygon(vehicles, city_polygon)
    
    # Plot the streets
    fig, ax = plt.subplots(figsize=(10, 10))
    streets = get_streets_within_polygon(city_polygon)
    streets.plot(ax=ax, color='gray', linewidth=0.5)
    
    # Plot the AED locations
    if not city_aeds.empty:
        city_aeds.plot(ax=ax, color='green', markersize=2, label='AED Locations')
    
    # Plot the card locations
    if not city_cards.empty:
        city_cards.plot(ax=ax, color='red', markersize=1, label='Cardiac Arrest Locations')
    
    # Plot the vehicle locations
    if not city_vehicles.empty:
        city_vehicles.plot(ax=ax, color='black', markersize=4, label='Vehicle Locations')
    
    ax.set_title(f"City: {city_name}")
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.set_aspect(1)
    ax.legend()
    plt.show()

In [104]:
print(len(brussels_points_gdf))
print(len(antwerpen_points_gdf))
print(len(liege_points_gdf))

688878
248709
234111
